In [166]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

np.random.seed(123)

In [167]:
# NETWORK = 'InceptionResNetV2'
# NETWORK = 'InceptionV3'
# NETWORK = 'Resnet'
# NETWORK = 'VGG16'
# NETWORK = 'VGG19'
# NETWORK = 'Xception'
# NETWORK = 'DenseNet201'
# NETWORK = 'EfficientNetB1'
# NETWORK = 'EfficientNetB4'
NETWORK = 'EfficientNetB7'
# NETWORK = 'NASNetLarge'
# NETWORK = 'ConvNeXtBase'

METHOD = "STD-Mean"

In [168]:
WORKING_DIRECTORY = '/datassd/WHOLEIMAGE_MAMIP'
PET_features_filename = f"Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"

features_path = os.path.join(WORKING_DIRECTORY, f"EXTRACTED_FEATURES/ExtractedFeatures_{METHOD}")
processed_features_path = os.path.join(WORKING_DIRECTORY, f"PROCESSED_FEATURES/ProcessedFeatures_{METHOD}")
if not os.path.exists(processed_features_path):
    os.makedirs(processed_features_path)


processed_features_filename = f"Processed_Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"
processed_features_full_filename = os.path.join(processed_features_path, processed_features_filename)

PET_featurs_full_path = os.path.join(features_path, PET_features_filename)

PET_filename = PET_features_filename.split(".")[0]

PET_outcome_data = pd.read_csv(PET_featurs_full_path)

outcome_file = "/datassd/WHOLEIMAGE_MAMIP/hecktor2022_patient_endpoint_training.csv"

In [169]:
PET_outcome_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,CHUM-001,0.325202,0.191769,0.697508,0.271463,0.719378,0.212432,0.254294,0.137993,0.475778,...,0.134622,0.579041,0.305964,0.136122,0.356942,0.349113,0.341881,0.302569,0.261420,0.576658
1,CHUM-002,0.317579,0.206237,0.771946,0.375914,0.569075,0.425038,0.533252,0.310801,0.308281,...,0.158278,0.221470,0.440222,0.415125,0.187008,0.155025,0.677216,0.288855,0.215743,0.867754
2,CHUM-006,0.312784,0.172533,0.649226,0.245868,0.552466,0.166965,0.545000,0.263302,0.563719,...,0.089841,0.470982,0.200168,0.132052,0.328719,0.272766,0.742846,0.185337,0.518589,1.027958
3,CHUM-007,0.359653,0.209176,0.726738,0.253937,0.403604,0.337837,0.547069,0.176843,0.578870,...,0.079670,0.302461,0.201798,0.211710,0.470151,0.241091,1.150393,0.068029,0.234310,0.895772
4,CHUM-008,0.190613,0.141235,0.752917,0.361011,0.478077,0.313439,0.562152,0.175124,0.338705,...,0.174821,0.473650,0.530384,0.103589,0.464655,0.225814,0.576358,0.149551,0.515543,0.547981


# Up to here

Read Outcome csv

In [170]:
OD = pd.read_csv(outcome_file)
OD = OD.iloc[:,:]
OD.head()

,PatientID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [171]:
OD = OD.rename(columns={'PatientID': 'Patient_ID'})
OD.head()

,Patient_ID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [172]:
OD.columns[0]

'Patient_ID'

In [173]:
first_column = PET_outcome_data.columns[0]
PET_outcome_data = PET_outcome_data.rename(columns={first_column: 'Patient_ID'})

In [174]:
OD.columns[0]


'Patient_ID'

In [175]:
PET_outcome_data.columns[0]

'Patient_ID'

In [176]:
PET_outcome_data.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,CHUM-001,0.325202,0.191769,0.697508,0.271463,0.719378,0.212432,0.254294,0.137993,0.475778,...,0.134622,0.579041,0.305964,0.136122,0.356942,0.349113,0.341881,0.302569,0.261420,0.576658
1,CHUM-002,0.317579,0.206237,0.771946,0.375914,0.569075,0.425038,0.533252,0.310801,0.308281,...,0.158278,0.221470,0.440222,0.415125,0.187008,0.155025,0.677216,0.288855,0.215743,0.867754
2,CHUM-006,0.312784,0.172533,0.649226,0.245868,0.552466,0.166965,0.545000,0.263302,0.563719,...,0.089841,0.470982,0.200168,0.132052,0.328719,0.272766,0.742846,0.185337,0.518589,1.027958
3,CHUM-007,0.359653,0.209176,0.726738,0.253937,0.403604,0.337837,0.547069,0.176843,0.578870,...,0.079670,0.302461,0.201798,0.211710,0.470151,0.241091,1.150393,0.068029,0.234310,0.895772
4,CHUM-008,0.190613,0.141235,0.752917,0.361011,0.478077,0.313439,0.562152,0.175124,0.338705,...,0.174821,0.473650,0.530384,0.103589,0.464655,0.225814,0.576358,0.149551,0.515543,0.547981


In [177]:
# ranges_list = list()
# for col in PET_outcome_data.iloc[:, 1:]:
#     col_range = PET_outcome_data[col].max() - PET_outcome_data[col].min()
#     ranges_list.append(col_range)
#     print(f"Range of column {col}: {col_range}")
# print(f"maximum range: {max(ranges_list)}")

In [178]:
full_features_df = pd.merge(PET_outcome_data, OD, on=OD.columns[0])

In [179]:
full_features_df.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,2552,2553,2554,2555,2556,2557,2558,2559,Relapse,RFS
0,CHUM-001,0.325202,0.191769,0.697508,0.271463,0.719378,0.212432,0.254294,0.137993,0.475778,...,0.305964,0.136122,0.356942,0.349113,0.341881,0.302569,0.261420,0.576658,0,1704
1,CHUM-002,0.317579,0.206237,0.771946,0.375914,0.569075,0.425038,0.533252,0.310801,0.308281,...,0.440222,0.415125,0.187008,0.155025,0.677216,0.288855,0.215743,0.867754,1,439
2,CHUM-006,0.312784,0.172533,0.649226,0.245868,0.552466,0.166965,0.545000,0.263302,0.563719,...,0.200168,0.132052,0.328719,0.272766,0.742846,0.185337,0.518589,1.027958,0,1186
3,CHUM-007,0.359653,0.209176,0.726738,0.253937,0.403604,0.337837,0.547069,0.176843,0.578870,...,0.201798,0.211710,0.470151,0.241091,1.150393,0.068029,0.234310,0.895772,0,1702
4,CHUM-008,0.190613,0.141235,0.752917,0.361011,0.478077,0.313439,0.562152,0.175124,0.338705,...,0.530384,0.103589,0.464655,0.225814,0.576358,0.149551,0.515543,0.547981,0,1499


In [180]:
full_features_df.to_csv(processed_features_full_filename, encoding='utf-8', index=False)